In [2]:
from nomad.metainfo import Quantity
from nomad.datamodel.metainfo.basesections import SynthesisMethod, ProcessStep
from nomad.datamodel import Schema
import numpy as np

In [8]:
import numpy as np
from typing import List, Union

from nomad.metainfo import (
    MSection, Quantity, SubSection, Section, Reference
)
from nomad.datamodel.metainfo.basesections import (
    ProcessStep, SynthesisMethod, CompositeSystemReference, InstrumentReference
)

from nomad.metainfo import MEnum

class TipType(MEnum):
    m_def = Section(a_enum=True)
    high_volume = 0
    low_volume = 1
    filtered = 2

class PipettePreparationSection(MSection):
    """Represents pipette preparation for dispensing solutions."""
    
    m_def = Section()
    
    pipette_rack_number = Quantity(
        type=int,
        description='Pipette rack number used for dispensing.'
    )
    
    tip_type = Quantity(
        type=TipType,
        description='Type of pipette tip used.'
    )
    
    solution_number = Quantity(
        type=int,
        description='Solution number to be dispensed.'
    )
    
    solution_volume = Quantity(
        type=float,
        unit='uL',
        description='Volume of solution to be dispensed.'
    )
    
    solvent_station_number = Quantity(
        type=int,
        description='Solvent station number for the solution.'
    )

class MixingSection(MSection):
    """Represents mixing parameters for solutions."""
    
    m_def = Section()
    
    mixing_cycles_aspiration = Quantity(
        type=int,
        description='Number of mixing cycles for aspiration (0-9).'
    )
    
    liquid_class_index_aspiration = Quantity(
        type=int,
        description='Liquid class index for aspiration.'
    )
    
    liquid_class_index_dispensing = Quantity(
        type=int,
        description='Liquid class index for dispensing.'
    )

class SpinParameters(MSection):
    """Common spin coating parameters."""
    
    m_def = Section()
    
    spin_acceleration = Quantity(
        type=float,
        unit='rpm/s',
        description='Acceleration rate of spin coater.'
    )
    
    spin_time = Quantity(
        type=float,
        unit='s',
        description='Duration of spinning.'
    )
    
    spin_velocity = Quantity(
        type=float,
        unit='rpm',
        description='Velocity of spin coater.'
    )
    
    spincoater_number = Quantity(
        type=int,
        description='Spincoater device number.'
    )

class AnnealingParameters(MSection):
    """Annealing parameters section."""
    
    m_def = Section()
    
    annealing_station_number = Quantity(
        type=int,
        description='Annealing station number.'
    )
    
    annealing_temperature = Quantity(
        type=float,
        unit='°C',
        description='Temperature for annealing.'
    )
    
    annealing_time = Quantity(
        type=float,
        unit='s',
        description='Duration of annealing.'
    )

class QuenchParameters(MSection):
    """Parameters for quenching step in spin coating."""
    
    m_def = Section()
    
    quench_pipette_rack_number = Quantity(
        type=int,
        description='Pipette rack number used for quenching.'
    )
    
    quench_tip_type = Quantity(
        type=TipType,
        description='Type of pipette tip used for quenching.'
    )
    
    quench_solution_number = Quantity(
        type=int,
        description='Quench solution number.'
    )
    
    quench_solution_volume = Quantity(
        type=float,
        unit='uL',
        description='Volume of quench solution.'
    )
    
    quench_solvent_station_number = Quantity(
        type=int,
        description='Solvent station number for quench solution.'
    )
    
    quench_height_above_substrate = Quantity(
        type=float,
        unit='mm',
        description='Height above substrate for quench dispensing.'
    )
    
    quench_liquid_class_index_aspiration = Quantity(
        type=int,
        description='Liquid class index for quench aspiration.'
    )
    
    quench_liquid_class_index_dispensing = Quantity(
        type=int,
        description='Liquid class index for quench dispensing.'
    )
    
    quench_spin_delay = Quantity(
        type=float,
        unit='s',
        description='Delay between quenching and second spin.'
    )

class SecondSpinParameters(MSection):
    """Parameters for second spin after quenching."""
    
    m_def = Section()
    
    second_spin_acceleration = Quantity(
        type=float,
        unit='rpm/s',
        description='Acceleration rate of second spin.'
    )
    
    second_spin_time = Quantity(
        type=float,
        unit='s',
        description='Duration of second spin.'
    )
    
    second_spin_velocity = Quantity(
        type=float,
        unit='rpm',
        description='Velocity of second spin.'
    )

class SpinCoatingStep(ProcessStep):
    """Base class for spin coating process steps."""
    
    m_def = Section()
    
    precursor = Quantity(
        type=Reference(CompositeSystemReference),
        description='Reference to the precursor material.'
    )
    
    solvent = Quantity(
        type=Reference(CompositeSystemReference),
        description='Reference to the solvent used.'
    )
    
    pipette_preparation = SubSection(
        section_def=PipettePreparationSection,
        description='Pipette preparation parameters.'
    )
    
    mixing = SubSection(
        section_def=MixingSection,
        description='Solution mixing parameters.',
        repeats=False,
        required=False
    )
    
    pipette_dispensing_height = Quantity(
        type=float,
        unit='mm',
        description='Height above substrate for dispensing.'
    )
    
    dispense_spin_blade_delay = Quantity(
        type=float,
        unit='s',
        description='Delay between dispensing and spin/blade operation.'
    )
    
    spin_parameters = SubSection(
        section_def=SpinParameters,
        description='Spin coating parameters.'
    )
    
    put_on_hotplate = Quantity(
        type=bool,
        description='Whether to put sample on hotplate after spinning.',
        required=False
    )
    
    annealing = SubSection(
        section_def=AnnealingParameters,
        description='Annealing parameters.',
        repeats=False,
        required=False
    )

class WhirlSpinCoatingStep(SpinCoatingStep):
    """Whirl spin coating process, with spinning before dispensing."""
    
    m_def = Section()
    
    initial_spin_velocity = Quantity(
        type=float,
        unit='rpm',
        description='Initial spin velocity before dispensing.'
    )
    
    arm_speed_while_dispensing = Quantity(
        type=float,
        unit='%',
        description='Arm speed while dispensing (0-100%).'
    )
    
    dispense_range = Quantity(
        type=float,
        unit='mm',
        description='Range for dispensing along arm path.'
    )

class PipetteQuenchingStep(SpinCoatingStep):
    """Spin coating with quenching step."""
    
    m_def = Section()
    
    quench_parameters = SubSection(
        section_def=QuenchParameters,
        description='Parameters for quenching step.'
    )
    
    second_spin = SubSection(
        section_def=SecondSpinParameters,
        description='Parameters for second spin after quenching.'
    )
    
    decap_and_recap_vial = Quantity(
        type=bool,
        description='Whether to decap and recap vial after process.',
        required=False
    )

class SolventMixingStep(ProcessStep):
    """Solvent mixing process step."""
    
    m_def = Section()
    
    aspirate_index = Quantity(
        type=int,
        description='Index for aspiration.'
    )
    
    aspirate_solution_number = Quantity(
        type=int,
        description='Solution number for aspiration.'
    )
    
    aspirate_solvent_station_number = Quantity(
        type=int,
        description='Solvent station number for aspiration.'
    )
    
    aspirate_volume = Quantity(
        type=float,
        unit='uL',
        description='Volume to aspirate.'
    )
    
    dispense_index = Quantity(
        type=int,
        description='Index for dispensing.'
    )
    
    dispense_solution_number = Quantity(
        type=int,
        description='Solution number for dispensing.'
    )
    
    dispense_solvent_station_number = Quantity(
        type=int,
        description='Solvent station number for dispensing.'
    )
    
    dispense_volume = Quantity(
        type=float,
        unit='uL',
        description='Volume to dispense.'
    )
    
    load_dispense_matrix = Quantity(
        type=bool,
        description='Whether to load dispense matrix.',
        required=False
    )
    
    pipette_rack_number = Quantity(
        type=int,
        description='Pipette rack number used.'
    )
    
    tip_type = Quantity(
        type=TipType,
        description='Type of pipette tip used.'
    )

class TakePhotoStep(ProcessStep):
    """Step to take a photo of the substrate."""
    
    m_def = Section()
    
    take_photo = Quantity(
        type=bool,
        description='Whether to take a photo.',
        default=True)
class PerovskiteFilmSynthesis(SynthesisMethod):
    """Complete perovskite film synthesis method."""
    
    m_def = Section()
    
    instruments = Quantity(
        type=Reference(InstrumentReference),
        shape=['*'],
        description='References to instruments used in synthesis.'
    )
    
    samples = Quantity(
        type=Reference(CompositeSystemReference),
        shape=['*'],
        description='References to samples produced.'
    )
    
    steps = SubSection(
        section_def=ProcessStep,
        description='Process steps in the synthesis.',
        repeats=True
    )

# Register the schema
if __name__ == "__main__":
    from nomad.datamodel import Schema
    schema = Schema(name='perovskite_synthesis', 
                   section_defs=[
                       TipType, PipettePreparationSection, MixingSection, 
                       SpinParameters, AnnealingParameters, QuenchParameters,
                       SecondSpinParameters, SpinCoatingStep, WhirlSpinCoatingStep,
                       PipetteQuenchingStep, SolventMixingStep, TakePhotoStep,
                       PerovskiteFilmSynthesis
                   ])
    description='Process steps in the synthesis.',
    repeats=True
    

AttributeError: 'NoneType' object has no attribute 'evaluate'